# Ask A Manager - Salary Survey

## Topics Covered

* CSV vs. Excel files
* Converting Strings to Integers
* String Methods in Action
* `.apply()` and `lambda`
* Using data from multiple DataFrame columns
* Data Preprocessing - Data Integration
* Pandas `.merge()`

## Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Our first dataset is a Google Sheet containing survey responses from employees in different management positions at different institutions, from 2021 to present. The survey was conducted as a [blog entry for Ask A Manger](https://www.askamanager.org/2021/04/how-much-money-do-you-make-4.html). The responses are collected in a Google Form and the questions are mostly short-form text response fields, with a few being multiple choice responses. This survey is real-world data, survey participants are all voluntary and self-selected. Not all the responses are recorded in the correct fields, and there is no standardization in the text fields pertaining to formatting numbers or abbreviating currencies or regions.

This dataset is a *mess*, and as such, it's perfect for practicing several data-cleaning techniques.

We are *NOT* going to clean the entire dataset today.

Let's say we want to analyze the salary range of the survey's participants, and get statistics for the median and mean salaries in the dataset. As we'll see, there are several obstacles in our way for accomplishing that goal.

### Data source:


"Ask A Manager" Salary Survey, 2021:

https://docs.google.com/spreadsheets/d/1IPS5dBSGtwYVbjsfbaMCYIWnOuRmJcbequohNxCyGVw/edit?resourcekey=&gid=1625408792#gid=1625408792

# EDA

## Differences between CSV and Excel Imports in Pandas

We have downloaded the data from Google Sheets in both .csv and .xlsx formats, to illustrate the differences between these formats in terms of how they are imported in pandas.

In [98]:
df_csv = pd.read_csv('https://raw.githubusercontent.com/ClaremontCollegesLibrary/PersnicketyPython/refs/heads/main/Ask%20A%20Manager%20Salary%20Survey%202021%20(Responses)%20-%20Form%20Responses%201.csv')

In [99]:
df_xl = pd.read_excel('https://raw.githubusercontent.com/ClaremontCollegesLibrary/PersnicketyPython/refs/heads/main/Ask%20A%20Manager%20Salary%20Survey%202021%20(Responses).xlsx')

In [100]:
df_csv.equals(df_xl)

False

### Why Aren't They The Same?

In [101]:
df_csv.head()

,Timestamp,How old are you?,What industry do you work in?,Job title,"If your job title needs additional context, please clarify here:","What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)","How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.",Please indicate the currency,"If ""Other,"" please indicate the currency here:","If your income needs additional context, please provide it here:",What country do you work in?,"If you're in the U.S., what state do you work in?",What city do you work in?,How many years of professional work experience do you have overall?,How many years of professional work experience do you have in your field?,What is your highest level of education completed?,What is your gender?,What is your race? (Choose all that apply.)
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


Before moving forward, let's unclutter those column names to make our comparison a bit more manageable. Part of the data-cleaning process is making quality-of-life adjustments, such as reducing on-screen clutter.

Take a moment to read the column names and absorb their meaning, then rename them to shorter column names in a consistent format.

We have chosen to label the columns with capitalized words, separated by underscores. Longer words have been abbreviated as a space consideration.

If you are just renaming a few column names instead of all of them, you can also use the pandas method in this format: `.rename(columns={"old_name1":"new_name1", "old_name2":"new_name2"})`)

Because we are renaming *all* of the columns, we can simply pass in a new list of the same length as the number of columns:

In [6]:
new_columns = [
    'Timestamp', 'Age', 'Industry', 'Job_Title', 'Addl_Context', 'Salary',
    'Addl_Comp', 'Currency', 'Curr_if_Other', 'Income_Context',
    'Country', 'US_State', 'City', 'Work_Exp', 'Field_Exp', 'Educ_Level',
    'Gender', 'Race'
]

df_csv.columns = new_columns
df_xl.columns = new_columns

These new names *mostly* capture the gist of the original questions. Please keep in mind that Country, US_State, and City all refer to the location of the job, not the location of residence.

In [7]:
df_csv.head()

,Timestamp,Age,Industry,Job_Title,Addl_Context,Salary,Addl_Comp,Currency,Curr_if_Other,Income_Context,Country,US_State,City,Work_Exp,Field_Exp,Educ_Level,Gender,Race
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


In [8]:
df_xl.head()

,Timestamp,Age,Industry,Job_Title,Addl_Context,Salary,Addl_Comp,Currency,Curr_if_Other,Income_Context,Country,US_State,City,Work_Exp,Field_Exp,Educ_Level,Gender,Race
0,2021-04-27 11:02:09.743,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,55000,0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,2021-04-27 11:02:21.562,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,54600,4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,2021-04-27 11:02:38.125,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,34000,NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,2021-04-27 11:02:40.643,25-34,Nonprofits,Program Manager,NaN,62000,3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,2021-04-27 11:02:41.793,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,60000,7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


The "Timestamp" column looks different. So does "Salary".

In [9]:
df_xl['Timestamp'].describe()

count                            28106
mean     2021-05-16 20:00:20.833426432
min         2021-04-27 11:02:09.743000
25%      2021-04-27 14:21:57.530749952
50%      2021-04-28 13:58:23.371000064
75%      2021-04-29 20:58:00.774749952
max         2025-01-16 10:00:46.663000
Name: Timestamp, dtype: object

In [10]:
df_csv['Timestamp'].describe()

count                  28106
unique                 25324
top       4/27/2021 12:05:06
freq                       5
Name: Timestamp, dtype: object

Using "==" on two DataFrames will return a Boolean value for *every cell*.

In [11]:
df_xl == df_csv

,Timestamp,Age,Industry,Job_Title,Addl_Context,Salary,Addl_Comp,Currency,Curr_if_Other,Income_Context,Country,US_State,City,Work_Exp,Field_Exp,Educ_Level,Gender,Race
0,False,True,True,True,False,False,True,True,False,False,True,True,True,True,True,True,True,True
1,False,True,True,True,False,False,True,True,False,False,True,False,True,True,True,True,True,True
2,False,True,True,True,False,False,False,True,False,False,True,True,True,True,True,True,True,True
3,False,True,True,True,False,False,True,True,False,False,True,True,True,True,True,True,True,True
4,False,True,True,True,False,False,True,True,False,False,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28101,False,True,True,True,False,False,False,True,False,False,True,True,True,True,True,True,True,True
28102,False,True,True,True,False,False,True,True,False,False,True,False,True,True,True,True,True,True
28103,False,True,True,True,False,False,False,True,False,False,True,True,True,True,True,True,True,True
28104,False,True,True,True,False,False,True,True,False,False,True,True,True,True,True,True,True,True


Some of the columns are identical, some of them have no common values, and some of them have some values that match and some that don't.

In [12]:
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28106 entries, 0 to 28105
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Timestamp       28106 non-null  object 
 1   Age             28106 non-null  object 
 2   Industry        28031 non-null  object 
 3   Job_Title       28105 non-null  object 
 4   Addl_Context    7273 non-null   object 
 5   Salary          28106 non-null  object 
 6   Addl_Comp       20792 non-null  float64
 7   Currency        28106 non-null  object 
 8   Curr_if_Other   211 non-null    object 
 9   Income_Context  3047 non-null   object 
 10  Country         28106 non-null  object 
 11  US_State        23072 non-null  object 
 12  City            28024 non-null  object 
 13  Work_Exp        28106 non-null  object 
 14  Field_Exp       28106 non-null  object 
 15  Educ_Level      27883 non-null  object 
 16  Gender          27935 non-null  object 
 17  Race            27929 non-null 

In [13]:
df_xl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28106 entries, 0 to 28105
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Timestamp       28106 non-null  datetime64[ns]
 1   Age             28106 non-null  object        
 2   Industry        28031 non-null  object        
 3   Job_Title       28105 non-null  object        
 4   Addl_Context    7273 non-null   object        
 5   Salary          28106 non-null  int64         
 6   Addl_Comp       20792 non-null  float64       
 7   Currency        28106 non-null  object        
 8   Curr_if_Other   211 non-null    object        
 9   Income_Context  3047 non-null   object        
 10  Country         28106 non-null  object        
 11  US_State        23072 non-null  object        
 12  City            28024 non-null  object        
 13  Work_Exp        28106 non-null  object        
 14  Field_Exp       28106 non-null  object        
 15  Ed

Notice that the counts of non-null objects are the same, but two of the columns show different data types: Timestamp and Salary are listed as "objects" in the .csv import, but `datetime64[ns]` and `int64` respectively in the xlsx import. The Pandas `read_excel()` method automatically reformats dates into a pandas timestamp, whereas `read_csv()` imports them literally as strings.

When evaluating Boolean expressions, `NaN` values in Python don't equal *anything*, even other `NaN` values.

In the original spreadsheet, many (but not all) of the `Salary` figures are formatted using commas to separate thousands.

`read_excel()` is built with this contingency in mind, and automatically converts numbers formatted this way into standard integers. `read_csv()` imports the entire column as strings regardless of their original formatting, because at least one of them has a comma in it.

Because of these differences, importing Excel files will usually be more reliable than importing csv files with respect to data integrity.

For this dataset, we *could* just use the imported Excel file rather than the csv file, because it would be easier to work with data that's already formatted the way we want.

However, you won't always have access to both csv and .xlsx versions of a spreadsheet in the wild. With that in mind, let's keep going.

In [14]:
#### Try this!

#df_csv['Salary'] = df_csv['Salary'].apply(int)

## Cleaning "Salary" using `.apply()` and `lambda`

The `int()` function won't work by itself to convert the Salary field from strings to integers, because of the commas. Unlike pandas `read_excel()`, `int()` isn't set up to handle commas automatically. Instead, we'll have to use a lambda function within our apply statement to provide a slightly more complex ad-hoc function:

As seen above, a quick way to create or modify new columns in a data frame is to use the `.apply()` method with and pass a function, often using the `lambda` keyword.

A Lambda function is an anonymous function (one that doesn't need to be named since it's used only in a local context). "Lambda" can mean different things in different programming languages, but in Python it allows for a quick ad hoc function to be used without needing to define the function with a `def` statement.

Using a lambda function allows us to apply a string method to a DataFrame column, assuming all the data are of the same type. In this case, all the items in our "Salary" column are strings, so we can use the `.replace()` method to clear out the commas, and then wrap the resulting string in an `int()` function to change the type to integer.

In [15]:
df_csv['Salary'] = df_csv['Salary'].apply(lambda x: int(x.replace(',','')))

In [16]:
#No more commas!
df_csv['Salary'].equals(df_xl['Salary'])

True

In [17]:
df_csv['Salary']

0          55000
1          54600
2          34000
3          62000
4          60000
          ...   
28101      75000
28102      25000
28103      72800
28104     100000
28105    1000000
Name: Salary, Length: 28106, dtype: int64

All clean! ...or is it?

### What do we need to remember about the Salary column?

Currency units!

In order to normalize the Salary column and produce summary statistics for it, we have to convert the totals to the same currency. We'll go with USD.

## More EDA

In [18]:
df_csv['Currency'].value_counts()

Currency
USD        23408
CAD         1675
GBP         1592
EUR          646
AUD/NZD      504
Other        164
CHF           37
SEK           37
JPY           23
ZAR           16
HKD            4
Name: count, dtype: int64

Wait... Australian Dollars and New Zealand Dollars aren't the same.

In [19]:
df_csv[df_csv['Currency'] == 'AUD/NZD']

,Timestamp,Age,Industry,Job_Title,Addl_Context,Salary,Addl_Comp,Currency,Curr_if_Other,Income_Context,Country,US_State,City,Work_Exp,Field_Exp,Educ_Level,Gender,Race
139,4/27/2021 11:05:51,35-44,Surveying,Accounts manager,NaN,85000,0.0,AUD/NZD,NaN,NaN,Australia,NaN,Perth,11 - 20 years,8 - 10 years,Master's degree,Man,White
470,4/27/2021 11:10:23,35-44,Health care,Doctor (Psychiatry registrar),NaN,134800,30000.0,AUD/NZD,NaN,First year training,Australia,NaN,Perth,8 - 10 years,2 - 4 years,PhD,Woman,Asian or Asian American
1188,4/27/2021 11:21:34,25-34,Education (Higher Education),Assistant professor,NaN,120000,20000.0,AUD/NZD,NaN,NaN,Australia,NaN,Perth,11 - 20 years,5-7 years,PhD,Man,Asian or Asian American
1362,4/27/2021 11:24:31,35-44,Education (early childhood),Early childhood educator,NaN,60000,0.0,AUD/NZD,NaN,NaN,Australia,NaN,Melbourne,21 - 30 years,21 - 30 years,College degree,Woman,"Hispanic, Latino, or Spanish origin"
2368,4/27/2021 11:42:58,35-44,Nonprofits,Field Coordinator,NaN,51600,NaN,AUD/NZD,NaN,NaN,Australia,NaN,Sydney,11 - 20 years,5-7 years,Master's degree,Woman,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27844,7/13/2022 20:19:39,18-24,Environmental science,Environmental Data Manager,NaN,70000,1000.0,AUD/NZD,NaN,NaN,Australia,NaN,Perth,2 - 4 years,2 - 4 years,College degree,Man,White
27951,4/1/2023 20:04:40,35-44,Education (Primary/Secondary),Teacher,NaN,104000,2000.0,AUD/NZD,NaN,Band 2 (of three) of the independent schools m...,Australia,NaN,"Maitland, NSW",5-7 years,5-7 years,College degree,Woman,White
27962,6/8/2023 4:35:52,45-54,Health care,Business Development Director,NaN,185000,22000.0,AUD/NZD,NaN,NaN,Australia,NaN,Sydney,21 - 30 years,1 year or less,College degree,Other or prefer not to answer,White
27997,11/30/2023 23:21:19,35-44,Sales,Business Development Manager,NaN,80000,45000.0,AUD/NZD,NaN,NaN,Australia,NaN,Melbourne,11 - 20 years,5-7 years,College degree,Woman,White


And that's only 11 currencies... what's in the "Other" category?

In [20]:
df_csv[df_csv['Currency'] == 'Other']

,Timestamp,Age,Industry,Job_Title,Addl_Context,Salary,Addl_Comp,Currency,Curr_if_Other,Income_Context,Country,US_State,City,Work_Exp,Field_Exp,Educ_Level,Gender,Race
434,4/27/2021 11:09:54,25-34,Computing or Tech,Data analyst,NaN,885000,0.0,Other,INR,NaN,India,NaN,Pune,2 - 4 years,2 - 4 years,Master's degree,Woman,Asian or Asian American
603,4/27/2021 11:12:24,45-54,Health care,Exec admin,Located in Argentina,1080000,223000.0,Other,Peso Argentino,Exchange about 1USD=165ARS,Argentina,NaN,City of Buenos Aires,11 - 20 years,11 - 20 years,College degree,Woman,"Hispanic, Latino, or Spanish origin"
1311,4/27/2021 11:23:39,25-34,Government and Public Administration,Associate,NaN,80640,20160.0,Other,MYR,NaN,Malaysia,NaN,Kuala Lumpur,5-7 years,2 - 4 years,College degree,Woman,Another option not listed here or prefer not t...
1840,4/27/2021 11:33:16,35-44,Intergovernmental organization,Consultant,NaN,60000,0.0,Other,CHF,"While I work full time at an organization, I a...",Switzerland,NaN,Geneva,11 - 20 years,5-7 years,Master's degree,Woman,Black or African American
1924,4/27/2021 11:34:55,35-44,Sales,CEO,NaN,800000,100000.0,Other,NOK,NaN,Norway,NaN,Oslo,11 - 20 years,11 - 20 years,College degree,Woman,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28061,8/20/2024 1:06:13,25-34,Health care,Clinical physiologist,NaN,1200000,NaN,Other,NGN,NaN,Nigeria,NaN,Lagos,2 - 4 years,2 - 4 years,College degree,Woman,Black or African American
28077,9/19/2024 19:16:28,25-34,Social Work,Counsellor,NaN,40000,0.0,Other,SGD,NaN,Singapore,NaN,Singapore,8 - 10 years,8 - 10 years,Master's degree,Woman,Asian or Asian American
28096,11/24/2024 8:08:37,25-34,Business or Consulting,BI Consultant,NaN,10,NaN,Other,Rupees,NaN,India,NaN,Noida,2 - 4 years,2 - 4 years,College degree,Man,Another option not listed here or prefer not t...
28100,12/11/2024 14:53:19,25-34,Health care,Pharmacist,NaN,24000000,2000000.0,Other,TZS,full time pharmacist and superintendent,Tanzania,NaN,Dar es Salaam,2 - 4 years,2 - 4 years,College degree,Woman,Black or African American


There are 504 rows where the currency is AUS/NZD... that's not a huge number relative to the size of our dataset, but we still don't want to throw out those entries; it would be best if we could salvage that data somehow.

### The Problem with AUS/NZD

There's a problem with the survey questionnaire itself: Australian Dollars and New Zealand Dollars are *not* equivalent.

We can try to solve this issue by splitting the AUD/NZD category based on the user's answer in the `Country` field. This may not work for 100% of the entries in question, but it will improve the accuracy of our summary statistics.

In [21]:
df_csv[df_csv['Currency'] == 'AUD/NZD']['Country'].unique().tolist()

['Australia ',
 'Australia',
 'New Zealand',
 'New Zealand ',
 'New Zealand Aotearoa',
 'New zealand',
 'NZ',
 'australia',
 ' New Zealand',
 'Australi',
 'Canada',
 'new zealand',
 'From New Zealand but on projects across APAC',
 'USA',
 'Aotearoa New Zealand']

We got mostly what we expected: variations on Australia and New Zealand with different spelling, capitalization, extra whitespace, etc.

However, there are three results that warrant further investigation: USA, Canada, and "From New Zealand but on projects across APAC".

In [22]:
df_csv[(df_csv['Currency'] == 'AUD/NZD') & (df_csv['Country'] == 'USA')]

,Timestamp,Age,Industry,Job_Title,Addl_Context,Salary,Addl_Comp,Currency,Curr_if_Other,Income_Context,Country,US_State,City,Work_Exp,Field_Exp,Educ_Level,Gender,Race
25510,5/6/2021 18:12:16,25-34,Transport or Logistics,Customer Service Rep/Dispatcher,NaN,30000,NaN,AUD/NZD,NaN,NaN,USA,New Jersey,Westampton,11 - 20 years,8 - 10 years,College degree,Woman,Black or African American


In [23]:
df_csv[(df_csv['Currency'] == 'AUD/NZD') & (df_csv['Country'] == 'Canada')]

,Timestamp,Age,Industry,Job_Title,Addl_Context,Salary,Addl_Comp,Currency,Curr_if_Other,Income_Context,Country,US_State,City,Work_Exp,Field_Exp,Educ_Level,Gender,Race
17426,4/28/2021 20:25:20,45-54,"Accounting, Banking & Finance",Manager,NaN,110000,10000.0,AUD/NZD,NaN,NaN,Canada,NaN,Ottawa,21 - 30 years,11 - 20 years,NaN,Woman,White


Neither the American nor the Canadian respondent indicates which country's currency they are paid in.

In [24]:
df_csv[df_csv['Country'] == 'From New Zealand but on projects across APAC']

,Timestamp,Age,Industry,Job_Title,Addl_Context,Salary,Addl_Comp,Currency,Curr_if_Other,Income_Context,Country,US_State,City,Work_Exp,Field_Exp,Educ_Level,Gender,Race
22585,4/30/2021 21:22:49,45-54,Business or Consulting,Workplace Strategist,NaN,180000,NaN,AUD/NZD,NaN,NaN,From New Zealand but on projects across APAC,NaN,Wellington,21 - 30 years,11 - 20 years,College degree,Woman,White


We're going to assume this last one is "NZD" as a default since the respondent lives in New Zealand, but the Salary figure may not represent 100% New Zealand Dollars. This is one entry out of around 28000.

### Removing Whitespace and Converting to Lower Case

Standardizing our survey responses will be easier if we can remove excess whitespace and convert all text to lower case:

In [25]:
df_csv['Country'] = df_csv['Country'].apply(lambda x: x.lower().strip())

While we're at it, we can also perform the same operation on Currency and Curr_if_Other:

In [26]:
df_csv['Currency'] = df_csv['Currency'].apply(lambda x: x.lower().strip())

What happens when we try to use this on the Curr_if_Other column?

In [27]:
#df_csv['Curr_if_Other'] = df_csv['Curr_if_Other'].apply(lambda x: x.lower().strip())

Null values (NaN) are treated as floating point numbers. This means we can't use string methods on them, and we'll have to find some means of bypassing them.

In [28]:
df_csv['Curr_if_Other'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 28106 entries, 0 to 28105
Series name: Curr_if_Other
Non-Null Count  Dtype 
--------------  ----- 
211 non-null    object
dtypes: object(1)
memory usage: 219.7+ KB


In [29]:
df_csv['Curr_if_Other'].fillna('', inplace=True)

C:\Users\davidm\AppData\Local\Temp\ipykernel_3776\3082733059.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_csv['Curr_if_Other'].fillna('', inplace=True)


In [30]:
df_csv['Curr_if_Other'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 28106 entries, 0 to 28105
Series name: Curr_if_Other
Non-Null Count  Dtype 
--------------  ----- 
28106 non-null  object
dtypes: object(1)
memory usage: 219.7+ KB


Now there are no null values in the Curr_if_Other column. Let's try that same `.apply()` method again.

In [31]:
df_csv['Curr_if_Other'] = df_csv['Curr_if_Other'].apply(lambda x: x.lower().strip())

In [32]:
au_nz_list = df_csv[df_csv['Currency'] == 'aud/nzd']['Country'].unique().tolist()
au_nz_list

['australia',
 'new zealand',
 'new zealand aotearoa',
 'nz',
 'australi',
 'canada',
 'from new zealand but on projects across apac',
 'usa',
 'aotearoa new zealand']

#### Python Code from Python Code

We now have a list of country names for respondents who are paid in "AUD/NZD". We can use this list to create a dictionary to further standardize the values in this field.

A neat little trick we can do in Python to avoid having to retype code is to write a chunk of code that prints out components of code that we can use later on.

For this, we can plug the list of "Country" values associated with AUD/NZD into a dictionary comprehension (yes, those exist!), and then use the printout as a template to insert the names we actually want to use.

In [33]:
au_nz_dict = {country: '' for country in au_nz_list}
au_nz_dict

{'australia': '',
 'new zealand': '',
 'new zealand aotearoa': '',
 'nz': '',
 'australi': '',
 'canada': '',
 'from new zealand but on projects across apac': '',
 'usa': '',
 'aotearoa new zealand': ''}

## CODING EXERCISE

In [34]:
#Copy-paste that here into the cell, replacing the "{}"
#then insert the values you think are appropriate to each country:

################################################################################
################################################################################


au_nz_dict = {}

au_nz_dict

################################################################################
################################################################################


{}

#### Finished Dictionary (Click to Reveal):

In [35]:
#Copy-paste that here, then insert the values appropriate to each country:

au_nz_dict = {'australia': 'australia',
 'new zealand': 'new zealand',
 'new zealand aotearoa': 'new zealand',
 'nz': 'new zealand',
 'australi': 'australia',
 'canada': 'canada',
 'from new zealand but on projects across apac': 'new zealand',
 'usa': 'usa',
 'aotearoa new zealand': 'new zealand'}

au_nz_dict

{'australia': 'australia',
 'new zealand': 'new zealand',
 'new zealand aotearoa': 'new zealand',
 'nz': 'new zealand',
 'australi': 'australia',
 'canada': 'canada',
 'from new zealand but on projects across apac': 'new zealand',
 'usa': 'usa',
 'aotearoa new zealand': 'new zealand'}

### Conditional Statents Inside Lambdas:

In [36]:
df_csv['Country'] = df_csv['Country'].apply(lambda x: au_nz_dict[x] if x in au_nz_dict.keys() else x)

In [37]:
How many values are there now?

Object `now` not found.


In [38]:
df_csv[df_csv['Currency'] == 'aud/nzd']['Country'].unique().tolist()

['australia', 'new zealand', 'canada', 'usa']

From here, how do we convert the Currency to AUD and NZD from AUD/NZD based on country?

We can use another `.apply(lambda)` statement, but this time, since we need to use data from both the Currency and Country columns, we have to define a function and use the `.apply()` on the DataFrame itself, not just a single column.

We do have two entries for which the Currency is listed as AUD/NZD but the Country is neither Australia nor New Zealand. We can check these for additional context, but we may need to end up throwing these out if we can't determine which Currency they are actually using. We'll convert these to "Other" for the time being, so we can dispense with the erroneous "AUD/NZD" label.

In [39]:
def currency_aud_nzd(x, y):
    """
    x = 'Currency', y = 'Country'

    Check if Currency is 'aud/nzd'
    If so, check Country
    
    """
    
    if x == 'aud/nzd':
        if y == 'australia':
            x = 'aud'
        elif y == 'new zealand':
            x = 'nzd'
        else:
            x = 'Other'
    
    return x

df_csv['Currency'] = df_csv.apply(lambda x: currency_aud_nzd(x['Currency'], x['Country']), axis=1)

In [40]:
df_csv['Currency'].unique()

array(['usd', 'gbp', 'cad', 'eur', 'aud', 'other', 'chf', 'zar', 'sek',
       'hkd', 'nzd', 'jpy', 'Other'], dtype=object)

In [41]:
df_csv['Curr_if_Other'].value_counts()

Curr_if_Other
                        27895
sgd                        13
dkk                        12
nok                        11
inr                        11
                        ...  
sek                         1
krw (korean won)            1
base plus commission        1
philippine peso             1
tzs                         1
Name: count, Length: 109, dtype: int64

That's a lot of distinct currencies in the Curr_if_Other column. We can also see that there are some entries that are not currencies, but rather comments or erroneously placed salary figures.

We will have to standardize the currency designations to match the three-character abbreviation, then merge the Currency and Curr_if_Other columns where Currency is 'Other'.

Let's see what values show up most and least often in the other fields:

In [42]:
len(df_csv['Country'].unique())

256

In [43]:
df_csv['Country'].value_counts().head(10)

Country
united states               10026
usa                          9065
us                           2779
canada                       1680
uk                            692
united kingdom                634
u.s.                          604
united states of america      492
australia                     390
germany                       197
Name: count, dtype: int64

In [44]:
df_csv['Country'].value_counts().tail(10)

Country
from romania, but for an us based company    1
somalia                                      1
wales (united kingdom)                       1
england, gb                                  1
danmark                                      1
u.k. (northern england)                      1
u.k                                          1
nl                                           1
nederland                                    1
česká republika                              1
Name: count, dtype: int64

In [45]:
df_csv['Job_Title'].value_counts().head(10)

Job_Title
Software Engineer           286
Project Manager             230
Director                    198
Senior Software Engineer    196
Program Manager             152
Teacher                     151
Librarian                   150
Manager                     145
Executive Assistant         136
Product Manager             125
Name: count, dtype: int64

In [46]:
df_csv['Job_Title'].value_counts().tail(10)

Job_Title
Tax Administrator                                 1
Pharmacy Supervisor                               1
Education and Outreach Coordinator                1
Associate Tutor                                   1
instructor of medicine                            1
Director of Agency Services                       1
Teacher - high school                             1
Payroll Administration                            1
Certified Coder                                   1
Product communication and education specialist    1
Name: count, dtype: int64

In [47]:
len(df_csv['Job_Title'].unique())/len(df_csv)

0.5114922080694514

In [48]:
df_csv['Age'].value_counts()

Age
25-34         12668
35-44          9907
45-54          3192
18-24          1236
55-64           994
65 or over       95
under 18         14
Name: count, dtype: int64

In [49]:
df_csv['Gender'].value_counts()

Gender
Woman                            21387
Man                               5502
Non-binary                         747
Other or prefer not to answer      298
Prefer not to answer                 1
Name: count, dtype: int64

In [50]:
df_csv['Work_Exp'].value_counts()

Work_Exp
11 - 20 years       9629
8 - 10 years        5381
5-7 years           4886
21 - 30 years       3644
2 - 4 years         3038
31 - 40 years        870
1 year or less       533
41 years or more     125
Name: count, dtype: int64

In [51]:
df_csv['Income_Context'].value_counts()

Income_Context
Hourly                                                                                                 4
Bonus not guaranteed                                                                                   3
Stock                                                                                                  3
10 month employee                                                                                      3
Before tax                                                                                             3
                                                                                                      ..
My income is salaried but weekly hourly expectations are 55-60 hours per week.                         1
I only work part time, so my actual take-home pay is closer to $40k/year.                              1
Additional compensation is calculated as % of my annual salary based on the company annual results     1
I'm paid minimum wage for my province   

In [52]:
df_csv['Field_Exp'].value_counts()

Field_Exp
11 - 20 years       6540
5-7 years           6524
2 - 4 years         6263
8 - 10 years        4984
21 - 30 years       1870
1 year or less      1500
31 - 40 years        384
41 years or more      41
Name: count, dtype: int64

In [53]:
df_csv['Educ_Level'].value_counts()

Educ_Level
College degree                        13536
Master's degree                        8877
Some college                           2075
PhD                                    1427
Professional degree (MD, JD, etc.)     1325
High School                             643
Name: count, dtype: int64

In [54]:
df_csv['US_State'].value_counts()

US_State
California                                 2611
New York                                   2174
Massachusetts                              1522
Texas                                      1269
Illinois                                   1213
                                           ... 
Alaska, Idaho, Oregon, Utah, Washington       1
Arizona, California, Nevada, Texas            1
Illinois, Kentucky                            1
Illinois, Wisconsin                           1
Florida, Georgia                              1
Name: count, Length: 137, dtype: int64

In [55]:
df_csv['City'].value_counts()

City
Boston                   772
Chicago                  752
New York                 711
Seattle                  691
London                   576
                        ... 
Ethel                      1
Concord, CA                1
charlottesville            1
A major Canadian city      1
Dar es Salaam              1
Name: count, Length: 4841, dtype: int64

In [56]:
df_csv

,Timestamp,Age,Industry,Job_Title,Addl_Context,Salary,Addl_Comp,Currency,Curr_if_Other,Income_Context,Country,US_State,City,Work_Exp,Field_Exp,Educ_Level,Gender,Race
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,55000,0.0,usd,,NaN,united states,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,54600,4000.0,gbp,,NaN,united kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,34000,NaN,usd,,NaN,us,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,62000,3000.0,usd,,NaN,usa,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,60000,7000.0,usd,,NaN,us,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28101,12/16/2024 19:10:58,18-24,Engineering or Manufacturing,Clinical Specialist - Biomedical Engineer,NaN,75000,NaN,usd,,NaN,united stares,New York,New York City,2 - 4 years,1 year or less,Master's degree,Woman,White
28102,12/20/2024 6:16:27,55-64,Government and Public Administration,HMRC Engagement Lead,NaN,25000,0.0,gbp,,NaN,united kingdom,NaN,Suffolk,21 - 30 years,2 - 4 years,Some college,Woman,White
28103,12/29/2024 23:50:26,25-34,Entertainment,Junior Editor,NaN,72800,NaN,usd,,NaN,united states,California,Los Angeles,5-7 years,5-7 years,Master's degree,Woman,"Hispanic, Latino, or Spanish origin, White"
28104,1/3/2025 16:03:10,18-24,Engineering or Manufacturing,Applications Engineer,NaN,100000,21000.0,usd,,NaN,usa,Texas,Dallas,1 year or less,1 year or less,Master's degree,Woman,White


### Reformatting Strings as Integers

In [57]:
df_csv.head()

,Timestamp,Age,Industry,Job_Title,Addl_Context,Salary,Addl_Comp,Currency,Curr_if_Other,Income_Context,Country,US_State,City,Work_Exp,Field_Exp,Educ_Level,Gender,Race
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,55000,0.0,usd,,NaN,united states,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,54600,4000.0,gbp,,NaN,united kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,34000,NaN,usd,,NaN,us,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,62000,3000.0,usd,,NaN,usa,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,60000,7000.0,usd,,NaN,us,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


In [58]:
df_csv['Addl_Comp'] = df_csv['Addl_Comp'].fillna(0)

In [59]:
df_csv['Currency'].value_counts()

Currency
usd      23408
cad       1675
gbp       1592
eur        646
aud        375
other      164
nzd        127
chf         37
sek         37
jpy         23
zar         16
hkd          4
Other        2
Name: count, dtype: int64

In [60]:
df_csv['Currency'] = df_csv['Currency'].apply(lambda x: x.lower())

In [61]:
df_csv['Curr_if_Other'].value_counts().head(50)

Curr_if_Other
                                                                                             27895
sgd                                                                                             13
dkk                                                                                             12
nok                                                                                             11
inr                                                                                             11
usd                                                                                             11
myr                                                                                              8
aud                                                                                              7
czk                                                                                              6
brl                                                                                            

## Data Preprocessing

In data science problems, columns in a dataset are sometimes referred to as "[features](https://en.wikipedia.org/wiki/Feature_(machine_learning))". Features are the discrete characteristics of a dataset. "Feature" is often used somewhat interchangeably with "explanatory variable"/"independent variable" to describe individual categories of data that serve as the input for a machine learning and/or statistical model.

Often, the existing features of a dataset are adequate but not optimal for creating a model. In such cases, it is sometimes possible to use relationships between different variables in one's dataset to create new features or update existing ones so they are more appropriate inputs. This is often called data transformation or "[feature engineering](https://en.wikipedia.org/wiki/Feature_engineering)". One may also pull in data from another source and combine it with the existing data. This process is referred to as data integration. 

All of these fall under the broader category of [data preprocessing](https://en.wikipedia.org/wiki/Data_preprocessing), which encompasses all the steps from loading the data, cleaning it, transforming it, and combining it with data from other sources, before the modeling stage.

In our dataset, we have Salary as a numerical feature and Currency as a categorical feature, but we can't really use either of them to get our summary statistics unless we combine them.

We also have a column for additional compensation, which reflects compensation not included in base salary. Not every respondent has a figure for this field, but we'd like to be able to use it where applicable.

Towards these goals, we need to find a means of converting currencies so we can interpret the salary all in one currency.

### Data Integration - Compustat Global Historical Currency Exchange Rates

When converting foreign currencies to US Dollars, it can be helpful to keep in mind that the exchange rates are not static. 

The Claremont Colleges Library has a subscription to Compustat Global, which offers historical currency conversions. A query to this database will allow us to get currency conversion rates for the time period in the dataset (2021-2025), and apply currency exchange rates appropriate to the timestamps recorded when participants of the survey responded.

Reference for abbreviations: https://www.foreignexchangelive.com/currency-codes-symbols/

After exporting the data from Compustat, we have an Excel file that contains currency abbreviations, date, and rate to US Dollars.

In [62]:
exch = pd.read_excel('https://raw.githubusercontent.com/ClaremontCollegesLibrary/PersnicketyPython/refs/heads/main/CompustatExchangeRates.xlsx')

In [63]:
exch

,Currency,Data Date,Rate to USD
0,AED,2021-01-01,0.272239
1,AFN,2021-01-01,0.012970
2,ALL,2021-01-01,0.009893
3,AMD,2021-01-01,0.001914
4,ANG,2021-01-01,0.559447
...,...,...,...
228391,XPF,2025-01-28,0.008730
228392,YER,2025-01-28,0.004020
228393,ZAR,2025-01-28,0.053459
228394,ZMW,2025-01-28,0.035688


To make merging the DataFrames easier, we can convert all the currency abbreviations to lower case.

In [64]:
exch['Currency'] = exch['Currency'].apply(lambda x: x.lower())

### List of Currency Abbreviations

In [65]:
currencies = exch['Currency'].unique().tolist()
currencies

['aed',
 'afn',
 'all',
 'amd',
 'ang',
 'aoa',
 'ars',
 'aud',
 'awg',
 'azm',
 'azn',
 'bam',
 'bbd',
 'bdt',
 'bgn',
 'bhd',
 'bif',
 'bmd',
 'bnd',
 'bob',
 'brl',
 'bsd',
 'btn',
 'bwp',
 'byn',
 'bzd',
 'cad',
 'cdf',
 'chf',
 'clf',
 'clp',
 'cnh',
 'cny',
 'cop',
 'crc',
 'cup',
 'cve',
 'czk',
 'djf',
 'dkk',
 'dop',
 'dzd',
 'ecs',
 'egp',
 'etb',
 'eur',
 'fjd',
 'gbp',
 'gel',
 'ghs',
 'gmd',
 'gnf',
 'gtq',
 'gyd',
 'hkd',
 'hnl',
 'hrk',
 'htg',
 'huf',
 'idr',
 'ils',
 'inr',
 'iqd',
 'irr',
 'isk',
 'jmd',
 'jod',
 'jpy',
 'kes',
 'kgs',
 'khr',
 'kmf',
 'kpw',
 'krw',
 'kwd',
 'kyd',
 'kzt',
 'lak',
 'lbp',
 'lkr',
 'lrd',
 'lsl',
 'lyd',
 'mad',
 'mdl',
 'mga',
 'mkd',
 'mmk',
 'mnt',
 'mop',
 'mru',
 'mur',
 'mvr',
 'mwk',
 'mxn',
 'myr',
 'mzn',
 'nad',
 'ngn',
 'nio',
 'nok',
 'npr',
 'nzd',
 'omr',
 'pab',
 'pen',
 'pgk',
 'php',
 'pkr',
 'pln',
 'pyg',
 'qar',
 'ron',
 'rsd',
 'rub',
 'rwf',
 'sar',
 'sbd',
 'scr',
 'sdg',
 'sek',
 'sgd',
 'sll',
 'sos',
 'srd',


In [66]:
df_csv['Curr_if_Other_clean'] = df_csv['Curr_if_Other'].fillna('').apply(lambda x: x.lower().strip())

In [67]:
df_csv['Curr_if_Other_clean']

0           
1           
2           
3           
4           
        ... 
28101       
28102       
28103       
28104       
28105    czk
Name: Curr_if_Other_clean, Length: 28106, dtype: object

### Currencies not in Data Dictionary

Given that there is no dropdown menu to select currency in the "Currency if Other" field of the survey, there will be some entries that do not conform to the standard three-character currency abbreviations.

In [68]:
currency_outliers = []

for currency in df_csv['Curr_if_Other_clean'].tolist():
    if currency not in currencies:
        if currency != '':
            currency_outliers.append(currency)
print("Nonstandard currency entries: ",len(currency_outliers))
currency_outliers

Nonstandard currency entries:  78


['peso argentino',
 '$76,302.34',
 'my bonus is based on performance up to 10% of salary',
 'i work for an online state university, managing admissions data. not direct tech support.',
 '0',
 'na',
 'br$',
 'base plus commission',
 'canadian',
 'indian rupees',
 'brl (r$)',
 'mexican pesos',
 'rsu / equity',
 'additonal = bonus plus stock',
 'american dollars',
 'pln (polish zloty)',
 'overtime (about 5 hours a week) and bonus',
 'czech crowns',
 'stock',
 'norwegian kroner (nok)',
 'ils/nis',
 '55,000',
 'aud & nzd are not the same currency...',
 'us dollar',
 'canadian',
 'nis (new israeli shekel)',
 '-',
 'rmb (chinese yuan)',
 'taiwanese dollars',
 "aud and nzd aren't the same currency, and have absolutely nothing to do with each other :(",
 'philippine peso',
 'krw (korean won)',
 'ils (shekel)',
 '6000 in stock grants annually',
 'china rmb',
 'aud australian',
 'polish złoty',
 'philippine peso (php)',
 'australian dollars',
 'many non-salary benefits - travel, free healthcare f

78 isn't a terribly high number of entries requiring correction, considering that there are 28106 entries in total. Still, this is a task you'll probably want to grab a cup of coffee or tea and put on some headphones for.

If we look at specific values, we can also find some records with erroneous data entries:



In [69]:
df_csv[df_csv['Curr_if_Other']=='0']

,Timestamp,Age,Industry,Job_Title,Addl_Context,Salary,Addl_Comp,Currency,Curr_if_Other,Income_Context,Country,US_State,City,Work_Exp,Field_Exp,Educ_Level,Gender,Race,Curr_if_Other_clean
1001,4/27/2021 11:18:27,35-44,Recruitment or HR,HR Manager/Accounts Payable,NaN,53500,0.0,usd,0,NaN,us,Minnesota,Minneapolis,11 - 20 years,2 - 4 years,College degree,Woman,White,0


In [70]:
df_csv[df_csv['Curr_if_Other']=='ekignkfb']

,Timestamp,Age,Industry,Job_Title,Addl_Context,Salary,Addl_Comp,Currency,Curr_if_Other,Income_Context,Country,US_State,City,Work_Exp,Field_Exp,Educ_Level,Gender,Race,Curr_if_Other_clean
28009,2/10/2024 15:04:31,25-34,"Accounting, Banking & Finance",mn jj,jmnjkkfd,15,0.0,usd,ekignkfb,dikfmklfb,dbfemf,"Alabama, Alaska, Arizona",dhgbfv,2 - 4 years,31 - 40 years,Some college,Woman,"Asian or Asian American, Black or African Amer...",ekignkfb


In [71]:
df_csv[df_csv['Curr_if_Other']=='rice']

,Timestamp,Age,Industry,Job_Title,Addl_Context,Salary,Addl_Comp,Currency,Curr_if_Other,Income_Context,Country,US_State,City,Work_Exp,Field_Exp,Educ_Level,Gender,Race,Curr_if_Other_clean
28021,3/20/2024 13:20:31,18-24,NaN,bum,supreme,10000000,299.0,usd,rice,ewad,usa,Alaska,Rice,41 years or more,8 - 10 years,"Professional degree (MD, JD, etc.)",Non-binary,White,rice


In [72]:
df_csv[df_csv['Curr_if_Other']=='55,000']

,Timestamp,Age,Industry,Job_Title,Addl_Context,Salary,Addl_Comp,Currency,Curr_if_Other,Income_Context,Country,US_State,City,Work_Exp,Field_Exp,Educ_Level,Gender,Race,Curr_if_Other_clean
9284,4/27/2021 17:36:26,45-54,Business or Consulting,Personal Executive Assistant,NaN,55,0.0,usd,"55,000",NaN,us,Michigan,Bloomfield Hills,21 - 30 years,11 - 20 years,College degree,Woman,White,"55,000"


In [73]:
df_csv[df_csv['Curr_if_Other']=='6000 in stock grants annually']

,Timestamp,Age,Industry,Job_Title,Addl_Context,Salary,Addl_Comp,Currency,Curr_if_Other,Income_Context,Country,US_State,City,Work_Exp,Field_Exp,Educ_Level,Gender,Race,Curr_if_Other_clean
11734,4/28/2021 4:27:20,45-54,Transport or Logistics,HR Manager,NaN,104917,17000.0,usd,6000 in stock grants annually,NaN,us,Kansas,Tooeka,21 - 30 years,11 - 20 years,Master's degree,Woman,White,6000 in stock grants annually


In [74]:
df_csv[df_csv['Curr_if_Other']=='47000']

,Timestamp,Age,Industry,Job_Title,Addl_Context,Salary,Addl_Comp,Currency,Curr_if_Other,Income_Context,Country,US_State,City,Work_Exp,Field_Exp,Educ_Level,Gender,Race,Curr_if_Other_clean
26531,7/6/2021 18:49:41,25-34,Nonprofits,Districtwide Program Coordinator,NaN,47000,0.0,usd,47000,NaN,united states,Michigan,Decatur,8 - 10 years,8 - 10 years,Master's degree,Woman,White,47000


## Dictionary for Currency Replacement

Below is the *beginning* of a dictionary that one could use to replace values in the original dataset. We're not going to replace every single value today, but this should give you an idea of the kind of fine-tuning one *can* do with a dataset like this.



It is not always necessary or advisable to clean a dataset entirely, as there is a trade-off between getting a task done perfectly and getting a task done at all. However, many practicing data scientists report that the majority of their time is spent cleaning data, as described in greater detail in this [Forbes article from 2016](https://www.forbes.com/sites/gilpress/2016/03/23/data-preparation-most-time-consuming-least-enjoyable-data-science-task-survey-says/).

In [75]:
currency_dict = {
    'canadian':'cad',
    'ntd':'twd',
    'danish kroner':'dkk',
    'converted mine into usd for your easyness':'usd',
    'php (philippine peso)':'php',
    'philippine pesos':'php',
    'argentine peso':'ars',
    'argentinian peso (ars)':'ars',
    'inr (indian rupee)':'inr',
    'israeli shekels':'ils',
    'it’s marketed as £22000 but we get paid pro-rats, so no pay for the school holidays.':'gbp'  
}

In [76]:
df_csv['Curr_if_Other_clean'] = df_csv['Curr_if_Other_clean'].apply(lambda x: currency_dict[x] if x in currency_dict.keys() else x)

In [77]:
df_csv['Curr_if_Other'].value_counts()

Curr_if_Other
                        27895
sgd                        13
dkk                        12
nok                        11
inr                        11
                        ...  
sek                         1
krw (korean won)            1
base plus commission        1
philippine peso             1
tzs                         1
Name: count, Length: 109, dtype: int64

We need to use multiple columns of our DataFrame in order to get all the currencies in one column. 

For this, we must define a new function that will let us replace "other" in the Currency column with whatever value is in the Curr_if_Other column, as we did before with AUD/NZD using both the Currency and Country columns.



In [78]:
def currency_combine(x, y):
    """
    x = Currency, y = Curr_if_Other_clean
    replaces currency == other with currency abbreviation from "Curr_if_Other_clean"    
    """
    
    value = x
    if value == 'other':
        value = y

    return value

df_csv['Currency_clean'] = df_csv.apply(lambda x: currency_combine(x['Currency'], x['Curr_if_Other_clean']), axis=1)

In [79]:
df_csv['Currency_clean'].value_counts()

Currency_clean
usd                 23416
cad                  1676
gbp                  1594
eur                   647
aud                   378
                    ...  
krw (korean won)        1
ils (shekel)            1
china rmb               1
aud australian          1
tzs                     1
Name: count, Length: 69, dtype: int64

# Merging the Datasets

In [80]:
exch['Data Date']

0        2021-01-01
1        2021-01-01
2        2021-01-01
3        2021-01-01
4        2021-01-01
            ...    
228391   2025-01-28
228392   2025-01-28
228393   2025-01-28
228394   2025-01-28
228395   2025-01-28
Name: Data Date, Length: 228396, dtype: datetime64[ns]

In [81]:
df_csv['Timestamp']

0         4/27/2021 11:02:10
1         4/27/2021 11:02:22
2         4/27/2021 11:02:38
3         4/27/2021 11:02:41
4         4/27/2021 11:02:42
                ...         
28101    12/16/2024 19:10:58
28102     12/20/2024 6:16:27
28103    12/29/2024 23:50:26
28104      1/3/2025 16:03:10
28105     1/16/2025 10:00:47
Name: Timestamp, Length: 28106, dtype: object

In [82]:
from datetime import datetime
import dateutil

In [83]:
#Format the original Timestamp string as a datetime object:
df_csv['Timestamp_YMD'] = df_csv['Timestamp'].apply(lambda x: dateutil.parser.parse(x))

#Reformat the resulting datetime object as a string that matches the currency exchange data format:
df_csv['Timestamp_YMD'] = df_csv['Timestamp_YMD'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d'))

In [84]:
df_csv['Timestamp_YMD']

0        2021-04-27
1        2021-04-27
2        2021-04-27
3        2021-04-27
4        2021-04-27
            ...    
28101    2024-12-16
28102    2024-12-20
28103    2024-12-29
28104    2025-01-03
28105    2025-01-16
Name: Timestamp_YMD, Length: 28106, dtype: object

In [85]:
df_csv.head()

,Timestamp,Age,Industry,Job_Title,Addl_Context,Salary,Addl_Comp,Currency,Curr_if_Other,Income_Context,...,US_State,City,Work_Exp,Field_Exp,Educ_Level,Gender,Race,Curr_if_Other_clean,Currency_clean,Timestamp_YMD
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,55000,0.0,usd,,NaN,...,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White,,usd,2021-04-27
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,54600,4000.0,gbp,,NaN,...,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White,,gbp,2021-04-27
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,34000,0.0,usd,,NaN,...,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White,,usd,2021-04-27
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,62000,3000.0,usd,,NaN,...,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White,,usd,2021-04-27
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,60000,7000.0,usd,,NaN,...,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White,,usd,2021-04-27


In [86]:
exch.head()

,Currency,Data Date,Rate to USD
0,aed,2021-01-01,0.272239
1,afn,2021-01-01,0.012970
2,all,2021-01-01,0.009893
3,amd,2021-01-01,0.001914
4,ang,2021-01-01,0.559447


In [87]:
exch.rename(columns={'Data Date':'Timestamp_YMD', "Currency":'Currency_clean'}, inplace=True)

In [88]:
exch['Timestamp_YMD'] = exch['Timestamp_YMD'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d'))

In [89]:
exch['Timestamp_YMD'].describe()

count         228396
unique          1488
top       2021-01-01
freq             154
Name: Timestamp_YMD, dtype: object

In [90]:
df_csv['Timestamp_YMD'].describe()

count          28106
unique           538
top       2021-04-27
freq           11257
Name: Timestamp_YMD, dtype: object

## Pandas `.merge()`

In [91]:
merged = pd.merge(left=df_csv, right=exch, how='left', on=['Currency_clean', 'Timestamp_YMD'])

In [92]:
merged

,Timestamp,Age,Industry,Job_Title,Addl_Context,Salary,Addl_Comp,Currency,Curr_if_Other,Income_Context,...,City,Work_Exp,Field_Exp,Educ_Level,Gender,Race,Curr_if_Other_clean,Currency_clean,Timestamp_YMD,Rate to USD
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,55000,0.0,usd,,NaN,...,Boston,5-7 years,5-7 years,Master's degree,Woman,White,,usd,2021-04-27,1.000000
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,54600,4000.0,gbp,,NaN,...,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White,,gbp,2021-04-27,1.392100
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,34000,0.0,usd,,NaN,...,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White,,usd,2021-04-27,1.000000
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,62000,3000.0,usd,,NaN,...,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White,,usd,2021-04-27,1.000000
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,60000,7000.0,usd,,NaN,...,Greenville,8 - 10 years,5-7 years,College degree,Woman,White,,usd,2021-04-27,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28101,12/16/2024 19:10:58,18-24,Engineering or Manufacturing,Clinical Specialist - Biomedical Engineer,NaN,75000,0.0,usd,,NaN,...,New York City,2 - 4 years,1 year or less,Master's degree,Woman,White,,usd,2024-12-16,1.000000
28102,12/20/2024 6:16:27,55-64,Government and Public Administration,HMRC Engagement Lead,NaN,25000,0.0,gbp,,NaN,...,Suffolk,21 - 30 years,2 - 4 years,Some college,Woman,White,,gbp,2024-12-20,1.257200
28103,12/29/2024 23:50:26,25-34,Entertainment,Junior Editor,NaN,72800,0.0,usd,,NaN,...,Los Angeles,5-7 years,5-7 years,Master's degree,Woman,"Hispanic, Latino, or Spanish origin, White",,usd,2024-12-29,1.000000
28104,1/3/2025 16:03:10,18-24,Engineering or Manufacturing,Applications Engineer,NaN,100000,21000.0,usd,,NaN,...,Dallas,1 year or less,1 year or less,Master's degree,Woman,White,,usd,2025-01-03,1.000000


In [93]:
merged['Rate to USD'].describe()

count    28065.000000
mean         1.005354
std          0.134037
min          0.000069
25%          1.000000
50%          1.000000
75%          1.000000
max          1.418300
Name: Rate to USD, dtype: float64

## The Payoff - Salary Summary Statistics

In [94]:
merged['Total_Comp'] = merged['Salary'] + merged['Addl_Comp']

In [95]:
merged['Salary_USD'] = merged['Salary'] * merged['Rate to USD']

In [96]:
merged['Total_Comp_USD'] = merged['Total_Comp'] * merged['Rate to USD']

In [97]:
merged[['Salary','Total_Comp','Salary_USD', 'Total_Comp_USD']].describe()

,Salary,Total_Comp,Salary_USD,Total_Comp_USD
count,2.810600e+04,2.810600e+04,2.806500e+04,2.806500e+04
mean,3.619435e+05,3.754410e+05,2.496746e+05,2.584640e+05
std,3.619466e+07,3.630425e+07,2.631055e+07,2.631055e+07
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.400000e+04,5.500000e+04,5.400000e+04,5.500000e+04
50%,7.500000e+04,7.900000e+04,7.500000e+04,7.759322e+04
75%,1.097000e+05,1.155000e+05,1.070000e+05,1.143000e+05
max,6.000070e+09,6.000070e+09,4.406588e+09,4.406588e+09


The summary statistics for "Salary" and "Total_Comp" are meaningless because the units aren't all the same. Adj_Salary_USD and Adj_Total_Comp_USD have been normalized to reflect the exchange rate from the original currency to USD at the time the survey was taken.

# What else could be done to improve the accuracy of these statistics?

### Other Resources:

The following are redundant links to the same dataset:

https://oscarbaruffa.com/messy/

https://www.r-bloggers.com/2021/04/a-real-world-messy-dataset-to-practice-on/


### Example Projects Using the "Ask A Manager" Dataset

A quick search yielded a few projects other people had done to analyze this dataset.

What are their goals? What do they do differently? How do they address the problem of cleaning the data?

https://annasanders.github.io/ms_projects/dtsa_5505/Data_Mining_Project_Report_final.pdf

https://github.com/brightboy373/Cleaning-and-Exploring-the-Ask-a-Manager-Survey-Dataset

https://github.com/maggiewolff/ask-a-manager-salary-survey

https://github.com/shaecodes/Ask-A-Manager

This one is done in R:

https://jtr13.github.io/cc19/ask-a-manager-salary-survey-dataset.html
